In [2]:
# %% [markdown]
"""
# ASL-GestureNet — Live Hand Tracking Demo (Jupyter Version)

This notebook-friendly version of `hand.py` runs a **live ASL hand-tracking demo**
using **MediaPipe Hands + MLP/CNN models**.

It:

- Captures webcam frames  
- Detects a single hand and draws landmarks + bounding box  
- Runs either **MLP** or **CNN** classifier on 21 keypoints  
- Shows the predicted ASL character on screen  
- Supports **caption mode** (auto/manual)  
- Supports **recording mode** for saving keypoint samples to CSV  

> ⚠️ **Note:** Live webcam (`cv.VideoCapture(0)`) may not work in some hosted
> notebook environments. It works best in a **local Jupyter** or **VS Code** setup.
"""
# %% [markdown]
"""
## 🧰 Keyboard Controls (Hotkeys)

### General
- `Q` — Quit program  
  - If recording is active, the CSV is auto-saved before exit  
- `X` — Clear on-screen caption  
- `V` — Toggle **caption mode** ON/OFF  
- `A` — When in caption mode, toggle between **AUTO** and **MANUAL** caption types  

### Caption Mode
- `SPACE`  
  - In **MANUAL** mode → append current letter (or space if none)  
- `ENTER` (`Return`)  
  - Save current caption to file and clear it  

### Recording Mode
- `R` — Start recording (you will be prompted for a CSV name)  
- `C` — Stop recording and save CSV  
- `SPACE` — Save current frame’s keypoints (while recording)  
- `F` — Change active label/class for recording
"""
# %%
# Core imports and configuration

from collections import deque
import statistics
import os
import warnings

import cv2 as cv
import mediapipe as mp
import absl.logging

from process import frame_process
from caption import (
    write_cap,
    write_title,
    box_pad,
    check_OOB,
    clear_cap,
    append_cap,
    save_cap_to_file,
)
from set_create import append_testdata, change_char, csv_startup, close_csv
from mlp_predictor import predict_asl_mlp
from cnn_predictor import predict_asl_cnn

# Silence noisy logs / warnings from TensorFlow + MediaPipe
warnings.filterwarnings(
    "ignore", message="SymbolDatabase.GetPrototype()", category=UserWarning
)
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
os.environ["MEDIAPIPE_DISABLE_GPU"] = "1"
absl.logging.set_verbosity(absl.logging.ERROR)

# Global toggle for recording mode
record_switch: bool = False


In [3]:
# %% [markdown]
"""
## ⚙️ Choose Model Type

Set `MODEL_TYPE` below:

- `1` → use MLP model (`predict_asl_mlp`)  
- `2` → use CNN model (`predict_asl_cnn`)  
"""
# %%
# Change this to 2 if you want to demo the CNN instead of the MLP
MODEL_TYPE = 1  # 1 = MLP, 2 = CNN


In [ ]:
# %% [markdown]
"""
## ▶️ Live Demo Function

Run `run_live_demo(MODEL_TYPE)` from the next cell to start the webcam loop.  
Press `Q` in the video window to exit.
"""
# %%
def run_live_demo(model_type: int = 1) -> None:
    """
    Run live webcam hand tracking loop.

    Args:
        model_type: 1 for MLP, 2 for CNN.
    """
    global record_switch

    if model_type not in (1, 2):
        print("Invalid model_type. Use 1 (MLP) or 2 (CNN).")
        return

    print(f"Starting ASL live demo with model_type = {model_type} ({'MLP' if model_type == 1 else 'CNN'})")

    # Rolling buffer to smooth predictions
    pred_buffer = deque(maxlen=5)

    kp_loop_count = 0

    # Open default webcam
    cap = cv.VideoCapture(0)
    cap.set(cv.CAP_PROP_FRAME_WIDTH, 1280)
    cap.set(cv.CAP_PROP_FRAME_HEIGHT, 720)

    if not cap.isOpened():
        print("!!! Error: Could not open video stream or file: WEBCAM REQUIRED FOR DEMO")
        return

    # MediaPipe Hands setup
    mp_drawing = mp.solutions.drawing_utils
    mp_hands = mp.solutions.hands
    hand = mp_hands.Hands(max_num_hands=1)  # single hand tracking

    # Displayed character
    asl_char = "?"

    # Caption mode state
    caption_mode = False
    caption_auto = False
    caption_text = ""  # (used via caption helpers)
    add_letter_time = 30
    countdown_timer = 0
    last_letter = None
    last_added = None
    clear_caption_time = 100
    space_counter = 0

    print("Webcam opened. Focus the 'Live Demo Feed' window and use hotkeys (Q, V, R, etc.).")

    while cap.isOpened():
        success, frame = cap.read()
        if not success:
            print("Frame grab failed, exiting loop.")
            break

        H, W = frame.shape[0], frame.shape[1]

        # MediaPipe expects RGB input
        RGB_frame = cv.cvtColor(frame, cv.COLOR_BGR2RGB)
        result = hand.process(RGB_frame)

        hand_coords: list[tuple[float, float]] = []
        hand_bbox_min = None
        hand_bbox_max = None

        # If at least one hand is detected
        if result.multi_hand_landmarks:
            for hand_landmarks in result.multi_hand_landmarks:
                # initialize bounding box extremes
                x_min = float("inf")
                y_min = float("inf")
                x_max = float("-inf")
                y_max = float("-inf")

                for lm in hand_landmarks.landmark:
                    # Normalized (x, y) from MediaPipe
                    cords = [lm.x, lm.y]
                    hand_coords.append(cords)

                    # Convert to pixel coords (avoid -1 index)
                    x = lm.x * (W - 1)
                    y = lm.y * (H - 1)

                    x_min = int(min(x_min, x))
                    y_min = int(min(y_min, y))
                    x_max = int(max(x_max, x))
                    y_max = int(max(y_max, y))

            hand_bbox_min = (x_min, y_min)
            hand_bbox_max = (x_max, y_max)

            # Add padding around detected hand box
            hand_bbox_min, hand_bbox_max = box_pad(
                hand_bbox_min, hand_bbox_max, 1.2
            )

            # Clamp bbox to image bounds
            hand_bbox_min = check_OOB(hand_bbox_min, H, W)
            hand_bbox_max = check_OOB(hand_bbox_max, H, W)

            # Determine if detected hand is left or right
            if (
                result.multi_handedness
                and result.multi_handedness[0].classification[0].label == "Right"
            ):
                is_left = False
            else:
                is_left = True

            # Process keypoints in the cropped region (normalization, debugging, etc.)
            frame_process(
                frame,
                hand_bbox_min,
                hand_bbox_max,
                hand_coords,
                is_left=is_left,
                display_hand_seg=True,
            )

            # Draw hand landmarks
            mp_drawing.draw_landmarks(
                frame,
                hand_landmarks,
                mp_hands.HAND_CONNECTIONS,
            )

            # Draw title and hand bbox label
            frame = write_title(frame, hand_bbox_min, hand_bbox_max, asl_char)

        # ===== PREDICTION (MLP / CNN) =====
        predicted_label = None
        smoothed_label = None

        if hand_coords:
            # Select which model to use
            if model_type == 1:
                predicted_label = predict_asl_mlp(hand_coords)
            else:
                predicted_label = predict_asl_cnn(hand_coords)

            pred_buffer.append(predicted_label)

            # Smooth out the predictions using mode of the buffer
            try:
                smoothed_label = statistics.mode(pred_buffer)
            except statistics.StatisticsError:
                smoothed_label = predicted_label

            if hand_bbox_min is not None and hand_bbox_max is not None:
                frame = write_title(frame, hand_bbox_min, hand_bbox_max, smoothed_label)

        # ===== HANDLE KEY INPUT (need this before caption logic) =====
        # Show bottom caption on the frame
        frame = write_cap(frame, W, H)
        cv.imshow("Live Demo Feed", frame)

        key = cv.waitKey(25) & 0xFF  # key pressed this frame

        # ========= CAPTION CONTROLS (main caption bar) =========
        # Append predicted label on Enter
        if key == ord("\r") and predicted_label is not None:
            append_cap(predicted_label)

        # Append space in caption (only if not in recording mode)
        if not record_switch and key == ord(" "):
            append_cap(" ")

        # Clear caption
        if key == ord("x") or key == ord("X"):
            clear_cap()

        # Toggle caption mode
        if key == ord("v") or key == ord("V"):
            clear_cap()
            caption_mode = not caption_mode
            print(f"CAPTION MODE {'ENABLED' if caption_mode else 'DISABLED'}")
            if caption_mode:
                print("CURRENT TYPE: MANUAL\nTO TOGGLE AUTO-CAPTION PRESS [A]")
            if caption_auto:
                print(
                    "CAPTION WILL BE UPDATED EVERY SECOND\n"
                    "AFTER 3 SECONDS OF NO HAND DETECTION THE CAPTION WILL SAVE AND CLEAR"
                )
            else:
                print(
                    "PRESS [SPACE] TO APPEND THE CURRENT LETTER (SPACE IF NONE)\n"
                    "AND [ENTER] TO SAVE AND CLEAR THE CURRENT CAPTION"
                )
            countdown_timer = 0
            last_letter = None

        # ========= CAPTION MODE (AUTO vs MANUAL) =========
        if caption_mode:
            # Toggle auto/manual caption behavior
            if key == ord("a") or key == ord("A"):
                caption_auto = not caption_auto
                print(
                    f"CURRENT CAPTION MODE: {'AUTO' if caption_auto else 'MANUAL'}"
                )
                if caption_auto:
                    print(
                        "CAPTION WILL BE UPDATED EVERY SECOND\n"
                        "AFTER 3 SECONDS OF NO HAND DETECTION THE CAPTION WILL SAVE AND CLEAR"
                    )
                else:
                    print(
                        "PRESS [SPACE] TO APPEND THE CURRENT LETTER (SPACE IF NONE)\n"
                        "AND [ENTER] TO SAVE AND CLEAR THE CURRENT CAPTION"
                    )

            if caption_auto:
                # AUTO CAPTION MODE
                if hand_coords and smoothed_label is not None:
                    current_letter = smoothed_label
                else:
                    current_letter = " "

                if current_letter == " " and last_letter is None:
                    last_letter = None
                    countdown_timer = 0
                elif current_letter != last_letter:
                    last_letter = current_letter
                    countdown_timer = 0

                if current_letter == last_letter and countdown_timer >= add_letter_time:
                    if current_letter != last_added:
                        if not (current_letter == last_added == " "):
                            append_cap(current_letter)
                            last_added = current_letter
                        write_cap(frame, W, H)
                        last_letter = current_letter

                if (
                    last_letter == current_letter == " "
                    and countdown_timer >= clear_caption_time
                ):
                    print("SAVING CURRENT CAPTION TO /captions")
                    save_cap_to_file()
                    clear_cap()
                    last_letter = None
                    countdown_timer = 0

                countdown_timer += 1

            else:
                # MANUAL CAPTION MODE
                if hand_coords and smoothed_label is not None:
                    current_letter = smoothed_label
                else:
                    current_letter = " "

                # SPACE appends the current letter
                if key == ord(" "):
                    append_cap(current_letter)

                write_cap(frame, W, H)

                # ENTER saves + clears caption
                if key == 13:  # Enter key
                    print("SAVING CURRENT CAPTION TO /captions")
                    save_cap_to_file()
                    clear_cap()
                    last_letter = None

        # ========= RECORDING CONTROLS =========
        # Enable frame recording hotkeys
        if key == ord("r") or key == ord("R"):
            if not record_switch:
                record_switch = True
                kp_loop_count = 0
                print(" 'R' pressed -> FRAME RECORDING BUTTON ENABLED")
                data_csv_name: str = input(
                    " > Provide name of csv you want to add to or create\n        > "
                )
                csv_startup(data_csv_name)
            else:
                print(" 'R' -> FRAME RECORDING BUTTON ALREADY ENABLED")

        # Disable frame recording hotkeys
        if record_switch and (key == ord("c") or key == ord("C")):
            record_switch = False
            close_csv()
            print(
                " 'C' -> FRAME RECORDING BUTTON DISABLED\n"
                "              SAVED CSV TO FILE :)"
            )

        # Hotkey to record a single frame's keypoint data
        if record_switch:
            if key == ord(" "):
                if hand_coords:
                    append_testdata(hand_coords)
                    kp_loop_count += 1

            if key == ord("f") or key == ord("F"):
                kp_loop_count = 0
                change_char()

            if kp_loop_count > 25:
                print(
                    "\n== 25 DATA POINTS CREATED FOR CLASS, INPUT NEXT_CHAR ==\n"
                )
                kp_loop_count = 0
                change_char()

        # ========= QUIT PROGRAM =========
        if key == ord("q") or key == ord("Q"):
            if record_switch:
                close_csv()
                print(
                    " ... 'Q' -> SAVED CSV FILE\n"
                    "             EXITING PROGRAM ..."
                )
                break
            print(" ... 'Q' -> EXITING PROGRAM ...")
            break

    cap.release()
    cv.destroyAllWindows()
    print("Demo finished. Webcam released, windows closed.")


In [6]:
# %% [markdown]
"""
## ▶️ Run the Demo

Run the cell below to start the live ASL demo using the model type
you set in `MODEL_TYPE` above.

> Remember: press **Q** in the video window to quit.
"""
# %%
run_live_demo(MODEL_TYPE)


Starting ASL live demo with model_type = 1 (MLP)
Webcam opened. Focus the 'Live Demo Feed' window and use hotkeys (Q, V, R, etc.).
 ... 'Q' -> EXITING PROGRAM ...
Demo finished. Webcam released, windows closed.
